# General Functions

In [1]:
def get_input(day):
    input_day = []
    with open(f"input_day_{day}", "r") as f:
        for line in f:
            input_day.append(line)
    return input_day

# [Day 1](https://adventofcode.com/2020/day/1)
## Part A
I wrote a solution that will look for an expected sum and return the values and the multiplication. If the sum cannot be build, it will just print that no solution exists and return None.

In [2]:
input_day_1 = [int(val) for val in get_input(1)]

In [3]:
def solution_part_1(input_list, expected_sum):
    input_list_sorted = sorted(input_list)
    start = 0
    end = len(input_list_sorted)-1
    try:
        while (int_sum:=input_list_sorted[start]+input_list_sorted[end])!=expected_sum:
            if int_sum<expected_sum:
                start += 1
            elif int_sum>expected_sum:
                end -= 1
    except IndexError:
        print("No valid solution found!")
        return None
    return (input_list_sorted[start], input_list_sorted[end]),input_list_sorted[start]*input_list_sorted[end]
solution_part_1(input_day_1, 2020)

((267, 1753), 468051)

## Part B
I rewrote the solution so it can solve Part A and B.

In [4]:
import itertools
import math
def get_day_1_result(input_list, amount_numbers, expected_sum):
    for combination in itertools.combinations(input_list, amount_numbers):
        if sum(combination)==expected_sum:
            return combination, math.prod(combination)
print(get_day_1_result(input_day_1, 3, 2020))

((523, 551, 946), 272611658)


## Speed Comparison
Comparing the Part A Solution and Part B Solution used for A.

In [5]:
%timeit get_day_1_result(input_day_1, 2, 2020)

3.27 ms ± 653 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
%timeit solution_part_1(input_day_1, 2020)

76.1 µs ± 18.6 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
